In [ ]:
import pandas as pd
import tempfile
import re
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import csv
from tqdm import tqdm_notebook as tqdm
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import lxml.html
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
pip install emoji

     |████████████████████████████████| 51kB 4.6MB/s 


In [ ]:
from emoji import UNICODE_EMOJI

In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
df_rec= pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')

In [ ]:
#df_ed = df_ed.dropna()
df_rec = df_rec.dropna()

In [ ]:
#df_ed.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_ed.csv')
df_rec.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')

NameError: ignored

In [ ]:
df_rec.head(10)

,Unnamed: 0,Unnamed: 0.1,users,content
0,0,0,thingsarestranger,Something positive you learned/experienced fro...
1,1,1,AnnaGreen3,Can we do something? Is there something we can...
2,2,2,taikutsuu,Please recognize that there is a realistic cha...
3,3,3,macespacee,Depression medication and appetite// weight lo...
4,4,4,raspbunni,reached a new hw :):):):) i stuffed myself lik...
5,5,5,probablyfuckedurdad,I wish people were more concerned about me. Is...
6,6,6,citrus_juuiice,"I ate normal for two days I feel so gross, I a..."
7,7,7,lovedieshere,Overeater finally became an undereater... So I...
8,8,8,illmurderyoudickhead,BMI of 21.29. Ashamed of myself. Title says it...
9,9,9,PoemOfLifeItself,you know that feeling when... you shit before ...


In [ ]:
#df_ed = pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_ed.csv')

In [ ]:
df_rec= pd.read_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/df_rec.csv')


## process

In [ ]:
def valid_text(text):
  if(text is None or text == '' or text == np.nan):
    return False
  else:
    return True
def remove_tags(text):
  if (valid_text(text)):
    if (lxml.html.fromstring(text).find('.//*') is not None):
      return re.sub('<[^>]*>', '', text)
    else:
      return text
def is_emoji(s):
  for emoji in UNICODE_EMOJI:
    if(emoji in str(s)):
      return True
    else:
      return False
def deEmojify(s):
  if(is_emoji(s)):
    return s.encode('ascii', 'ignore').decode('ascii')
  else:
    return s
#Split words
def splitting(text):
  tokens = ""
  try:
    tokens = word_tokenize(text)
  except:
    if(text == np.nan):
      print("nan")
    print("not nan")
  return tokens
def remove_Nones(words):
  text = []
  for w in words:
    if (w != 'None'):
      text.append(w)
  return text
def delinks(words):
  #text = [re.sub(r'''(?i)\b((?:(https|http)?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', w, flags=re.MULTILINE) for w in words]
  text = [re.sub(r"http\S+", "", w) for w in words]
  return text
def remove_numbers(words):
  output = [re.sub(r'\d+', '', w) for w in words]
  return output
def depunct(tokens):
  # remove all tokens that are not alphabetic
  words = [word for word in tokens if word.isalpha()]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in words]
  return stripped
def de_stopwords(words):
  stop_words = stopwords.words('english')
  stop_words.extend(['http', 'lw', 'gw', 'ugw', 'f', 'u'])
  l = [w for w in words if not w in stop_words]
  return l
def tolowercase(words):
  text = [w.lower() for w in words]
  return text
def stemming(tokens):
  porter = PorterStemmer()
  stemmed = [porter.stem(word) for word in tokens]
  return stemmed
def load_texts(text):
  inp = deEmojify(text)
  return splitting(inp)
def pre_processing(l):
  no_none = remove_Nones(l)
  no_link = delinks(no_none)
  no_numb = remove_numbers(no_link)
  return stemming(de_stopwords(tolowercase(depunct(no_numb))))
def dataframe_preprocessing(df):
  contents = []
  users = []
  for ind, row in tqdm(df.iterrows(), total=len(df)):
    users.append(row["users"])
    #l = pre_processing(load_texts(row["content"]))
    txt = load_texts(row["content"])
    contents.append(pre_processing(txt))
  return users, contents

## execute

In [ ]:
#u_ed, c_ed = dataframe_preprocessing(df_ed)

In [ ]:
u_rec, c_rec = dataframe_preprocessing(df_rec)

#Algorithm (LDA)

#Bag of words

In [ ]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [ ]:
#dictionary_ed = gensim.corpora.Dictionary(c_ed)

NameError: ignored

In [ ]:
type(c_rec[0])

list

In [ ]:
dictionary_rec = gensim.corpora.Dictionary(c_rec)

In [ ]:
#dictionary_ed.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
dictionary_rec.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
#bow_corpus_ed = [dictionary_ed.doc2bow(doc) for doc in c_ed]

NameError: ignored

In [ ]:
bow_corpus_rec = [dictionary_rec.doc2bow(doc) for doc in c_rec]

In [ ]:
dictionary_rec[6]

'anoth'

#LDA using bag of words

In [ ]:
'''lda_model_ed = gensim.models.ldamodel.LdaModel(bow_corpus_ed, num_topics=9, id2word=dictionary_ed, random_state=100, update_every=1, chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)'''

KeyboardInterrupt: ignored

In [ ]:
lda_model_rec = gensim.models.ldamodel.LdaModel(bow_corpus_rec, num_topics=9, id2word=dictionary_rec, random_state=100, update_every=1, chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

KeyboardInterrupt: ignored

In [ ]:
'''for idx, topic in lda_model_ed.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))'''

In [ ]:
for idx, topic in lda_model_rec.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.121*"post" + 0.073*"sub" + 0.062*"rule" + 0.036*"ban" + 0.034*"proedmem" + 0.031*"subreddit" + 0.026*"pro" + 0.025*"support" + 0.024*"comment" + 0.023*"commun"
Topic: 1 
Words: 0.026*"feel" + 0.017*"weight" + 0.017*"day" + 0.016*"eat" + 0.014*"fuck" + 0.011*"want" + 0.011*"get" + 0.011*"much" + 0.011*"go" + 0.010*"bing"
Topic: 2 
Words: 0.016*"peopl" + 0.014*"think" + 0.014*"know" + 0.013*"get" + 0.013*"ed" + 0.011*"realli" + 0.011*"eat" + 0.011*"feel" + 0.009*"thing" + 0.009*"say"
Topic: 3 
Words: 0.102*"amp" + 0.043*"xb" + 0.035*"carbohydr" + 0.031*"insulin" + 0.027*"syrup" + 0.022*"oz" + 0.020*"mango" + 0.014*"muscl" + 0.013*"latt" + 0.012*"starbuck"
Topic: 4 
Words: 0.049*"girl" + 0.041*"look" + 0.039*"thin" + 0.031*"love" + 0.029*"beauti" + 0.025*"men" + 0.024*"blog" + 0.023*"skinni" + 0.020*"women" + 0.019*"guy"
Topic: 5 
Words: 0.023*"calori" + 0.022*"eat" + 0.020*"food" + 0.011*"use" + 0.008*"also" + 0.008*"count" + 0.007*"good" + 0.007*"carb" + 0.006*"one" +

#LDA visualization

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [ ]:
import pyLDAvis


##Bag of words

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts, users):
    # Init output
    topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topics_df = topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    redditors = pd.Series(users)
    topics_df = pd.concat([topics_df, contents, redditors], axis=1)
    return(topics_df)

In [ ]:
'''df_topic_keywords_ed = format_topics_sentences(lda_model_ed, bow_corpus_ed, c_ed, u_ed)

# Format
df_dominant_topic_ed = df_topic_keywords_ed.reset_index()
df_dominant_topic_ed.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']'''

In [ ]:
#df_dominant_topic_rec.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Redditors
0,0,2.0,0.6183,"like, get, feel, really, know, even, weight, m...","[something, positive, proed, always, considere...",thingsarestranger
1,1,2.0,0.7615,"like, get, feel, really, know, even, weight, m...","[something, something, recover, like, explaini...",AnnaGreen3
2,2,2.0,0.7647,"like, get, feel, really, know, even, weight, m...","[please, recognize, realistic, chance, sub, ta...",taikutsuu
3,3,2.0,0.7908,"like, get, feel, really, know, even, weight, m...","[depression, medication, weight, loss, gain, r...",macespacee
4,4,2.0,0.7765,"like, get, feel, really, know, even, weight, m...","[reached, new, hw, stuffed, like, fat, pig, pr...",raspbunni
5,5,2.0,0.7570,"like, get, feel, really, know, even, weight, m...","[wish, people, concerned, selfish, told, paren...",probablyfuckedurdad
6,6,2.0,0.7790,"like, get, feel, really, know, even, weight, m...","[ate, normal, two, days, feel, gross, ate, lik...",citrus_juuiice
7,7,2.0,0.7793,"like, get, feel, really, know, even, weight, m...","[overeater, finally, became, undereater, ednos...",lovedieshere
8,8,2.0,0.6014,"like, get, feel, really, know, even, weight, m...","[bmi, ashamed, title, says, even, know, feel, ...",illmurderyoudickhead
9,9,2.0,0.7174,"like, get, feel, really, know, even, weight, m...","[know, feeling, shit, weigh, discord, anyone, ...",PoemOfLifeItself


In [ ]:
#df_dominant_topic_ed.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/topics/df_topics_ed.csv')

In [ ]:
'''import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_ed, bow_corpus_ed, dictionary=lda_model_ed.id2word)
vis'''

ModuleNotFoundError: ignored

## pro recovery

In [ ]:
df_topic_keywords_rec = format_topics_sentences(lda_model_rec, bow_corpus_rec, c_rec, u_rec)

# Format
df_dominant_topic_rec = df_topic_keywords_rec.reset_index()
df_dominant_topic_rec.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

In [ ]:
df_dominant_topic_rec.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Redditors
0,0,2.0,0.3735,"peopl, think, know, get, ed, realli, eat, feel...","[someth, posit, pro, alway, consid, lax, form,...",thingsarestranger
1,1,2.0,0.5074,"peopl, think, know, get, ed, realli, eat, feel...","[someth, someth, recov, like, explain, reddit,...",AnnaGreen3
2,2,1.0,0.3264,"feel, weight, day, eat, fuck, want, get, much,...","[pleas, recogn, realist, chanc, sub, taken, ye...",taikutsuu
3,3,1.0,0.4764,"feel, weight, day, eat, fuck, want, get, much,...","[depress, medic, weight, loss, gain, realli, b...",macespacee
4,4,2.0,0.3369,"peopl, think, know, get, ed, realli, eat, feel...","[reach, new, hw, stuf, like, fat, pig, pregnan...",raspbunni
5,5,2.0,0.4707,"peopl, think, know, get, ed, realli, eat, feel...","[wish, peopl, concern, selfish, told, parent, ...",probablyfuckedurdad
6,6,1.0,0.3889,"feel, weight, day, eat, fuck, want, get, much,...","[ate, normal, two, day, feel, gross, ate, like...",citrus_juuiice
7,7,1.0,0.4539,"feel, weight, day, eat, fuck, want, get, much,...","[overeat, final, becam, undereat, edno, year, ...",lovedieshere
8,8,2.0,0.5544,"peopl, think, know, get, ed, realli, eat, feel...","[bmi, asham, titl, say, even, know, feel, asha...",illmurderyoudickhead
9,9,1.0,0.4470,"feel, weight, day, eat, fuck, want, get, much,...","[know, feel, shit, weigh, discord, anyon, link...",PoemOfLifeItself


In [ ]:
df_dominant_topic_rec.to_csv(r'/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_9/topics/df_topics_rec.csv')

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis_rec = pyLDAvis.gensim.prepare(lda_model_rec, bow_corpus_rec, dictionary=lda_model_rec.id2word)
vis_rec

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.281777 -0.042357       1        1  35.567814
1      0.302082 -0.037213       2        1  28.547457
5      0.057450  0.165025       3        1  14.709113
8      0.227299  0.039632       4        1  14.176835
6     -0.185903  0.252148       5        1   1.977163
4     -0.119050 -0.284276       6        1   1.969923
3     -0.208018  0.014444       7        1   1.069290
0     -0.196183 -0.097975       8        1   1.065015
7     -0.159454 -0.009428       9        1   0.917392, topic_info=     Category          Freq        Term         Total  loglift  logprob
129   Default  11351.000000        look  11351.000000  30.0000  30.0000
162   Default   4660.000000       pleas   4660.000000  29.0000  29.0000
459   Default   5219.000000        diet   5219.000000  28.0000  28.0000
319   Default   6332.000000        love   6332.000000  27.0000  27.0000
676   Default   3908.000000       thank   3908.000000  26.0000  26.0000
351   Default   8405.000000         see   8405.000000  25.0000  25.0000
58    Default   5053.000000       drink   5053.000000  24.0000  24.0000
165   Default   3666.000000        post   3666.000000  23.0000  23.0000
1     Default   3265.000000         add   3265.000000  22.0000  22.0000
82    Default  25392.000000        feel  25392.000000  21.0000  21.0000
31    Default   8452.000000      calori   8452.000000  20.0000  20.0000
436   Default   4300.000000     comment   4300.000000  19.0000  19.0000
334   Default  12913.000000       peopl  12913.000000  18.0000  18.0000
84    Default   5021.000000        find   5021.000000  17.0000  17.0000
250   Default   3351.000000       water   3351.000000  16.0000  16.0000
390   Default   2465.000000         amp   2465.000000  15.0000  15.0000
90    Default  11909.000000        food  11909.000000  14.0000  14.0000
51    Default  16206.000000         day  16206.000000  13.0000  13.0000
93    Default   8839.000000        fuck   8839.000000  12.0000  12.0000
96    Default  24428.000000         get  24428.000000  11.0000  11.0000
466   Default  10452.000000          ed  10452.000000  10.0000  10.0000
503   Default   2656.000000        girl   2656.000000   9.0000   9.0000
794   Default   2708.000000        thin   2708.000000   8.0000   8.0000
713   Default   2593.000000        zero   2593.000000   7.0000   7.0000
254   Default  17848.000000      weight  17848.000000   6.0000   6.0000
355   Default   2797.000000      skinni   2797.000000   5.0000   5.0000
78    Default  11423.000000         fat  11423.000000   4.0000   4.0000
218   Default   1757.000000         sub   1757.000000   3.0000   3.0000
434   Default   1982.000000        coke   1982.000000   2.0000   2.0000
543   Default   3466.000000          lb   3466.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
4529   Topic9     60.527069       edibl     66.830933   4.5923  -5.8307
4201   Topic9     64.856712      instal     72.330170   4.5823  -5.7616
5515   Topic9     24.312046          tm     29.112494   4.5112  -6.7428
4189   Topic9     29.570389        cozi     35.576344   4.5065  -6.5470
5374   Topic9     19.183491    platform     23.982264   4.4681  -6.9798
3043   Topic9     11.334747         kik     16.474846   4.3174  -7.5059
2687   Topic9     44.852699     shatter     66.767235   4.2936  -6.1304
1656   Topic9    385.240814     account    652.008728   4.1652  -3.9799
1376   Topic9    845.465576          hi   1508.631348   4.1123  -3.1939
951    Topic9    373.798828          ad    727.599365   4.0254  -4.0101
5283   Topic9      5.497988      reread     11.067611   3.9918  -8.2294
4748   Topic9     17.638193    commonli     37.276562   3.9431  -7.0637
4917   Topic9      4.052139      server      8.851515   3.9100  -8.5346
676    Topic9   1770.888550       thank   3908.097168   3.8998  -2.4546
5640   Topic9     15.794687        glam     35.3848

coherence scores:

In [ ]:
'''coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=c_ed, dictionary=dictionary_ed, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)'''

KeyboardInterrupt: ignored

In [ ]:
'''coherence_model_lda = CoherenceModel(model=lda_model_ed, texts=c_ed, dictionary=dictionary_ed, coherence="u_mass")
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)'''


Coherence Score:  -0.7402897378539399


rec

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_rec, texts=c_rec, dictionary=dictionary_rec, coherence='c_v')
coherence_lda_rec = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda_rec)


Coherence Score:  0.34070560136118055


In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model_rec, texts=c_rec, dictionary=dictionary_rec, coherence='u_mass')
coherence_lda_rec = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda_rec)


Coherence Score:  -1.6664061478257466


In [ ]:
from gensim.models import HdpModel

In [ ]:
hdp = HdpModel(bow_corpus_rec, dictionary_rec)

In [ ]:
topic_info = hdp.print_topics(num_topics=20, num_words=10)

In [ ]:
for idx, topic in hdp.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*like + 0.009*feel + 0.007*weight + 0.007*get + 0.007*know + 0.006*eat + 0.006*really + 0.006*eating + 0.006*people + 0.006*even
Topic: 1 
Words: 0.015*like + 0.009*feel + 0.007*really + 0.007*know + 0.007*weight + 0.007*get + 0.007*eat + 0.007*eating + 0.006*food + 0.006*much
Topic: 2 
Words: 0.012*like + 0.007*eat + 0.006*feel + 0.006*food + 0.006*get + 0.006*eating + 0.005*one + 0.005*much + 0.005*know + 0.005*want
Topic: 3 
Words: 0.010*like + 0.006*weight + 0.005*get + 0.005*really + 0.005*feel + 0.005*one + 0.005*even + 0.004*know + 0.004*think + 0.004*people
Topic: 4 
Words: 0.013*like + 0.007*feel + 0.005*food + 0.005*eating + 0.005*eat + 0.005*get + 0.005*know + 0.005*really + 0.004*much + 0.004*people
Topic: 5 
Words: 0.011*like + 0.007*would + 0.006*weight + 0.006*get + 0.006*eating + 0.006*even + 0.005*feel + 0.005*eat + 0.005*really + 0.005*people
Topic: 6 
Words: 0.009*like + 0.004*feel + 0.004*weight + 0.004*get + 0.004*time + 0.004*one + 0.004*much

In [ ]:
def format_topics_sentences(hdp, corpus, texts, users):
    # Init output
    topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(hdp[corpus]):
        row = row_list[0] if hdp.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                topics_df = topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    redditors = pd.Series(users)
    topics_df = pd.concat([topics_df, contents, redditors], axis=1)
    return(topics_df)

In [ ]:
df_topic_keywords_rec = format_topics_sentences(hdp, bow_corpus_rec, c_rec, u_rec)

# Format
df_dominant_topic_rec = df_topic_keywords_rec.reset_index()
df_dominant_topic_rec.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Redditors']

AttributeError: ignored